For WoRMS in the SGCN case, we are only using it as a source for additional taxonomic name matching, although there is additional data available from WoRMS such as species traits that we could use at some future point. Because of this, we only need to pick up the unmatched names from the ITIS cache in order to build the messages for a WoRMS processing queue. A WoRMS message is sent to the cache each time the ITIS process runs and does not find a valid name. This workflow operates the WoRMS processor on all of those messages. For valid WoRMS records, the process sends summary properties to the message queue used to infuse those properties into the SGCN master table to put species onto the "SGCN National List."

In [1]:
import pysgcn
sgcn = pysgcn.sgcn.Sgcn()

from joblib import Parallel, delayed
from tqdm import tqdm

mq = "mq_worms_check"
sppin_source = "worms"

In [2]:
messages = sgcn.sql_mq.get_all_records("mq", mq)
print(len(messages))
messages[0]

2355


{'id': 'e4fd04dbfeb8446265f178087d1c0fa7c6e1e56c',
 'date_inserted': '2019-12-19T13:22:43.285527',
 'body': {'source': {'type': 'List of Scientific Names',
   'name_source': 'ITIS Search'},
  'sppin_key': 'Scientific Name:Scaphytopius dorsalis'}}

In [3]:
%%time
Parallel(n_jobs=8, prefer="threads")(
    delayed(sgcn.process_sppin_source_search_term)
    (
        message_queue=mq,
        sppin_source=sppin_source,
        message_id=message["id"], 
        message_body=message["body"]
    ) for message in tqdm(messages)
)

  2%|▏         | 40/2355 [00:04<04:25,  8.72it/s]

OperationalError: table worms has no column named data